In [119]:
def accuracy(true, predicted):
    return np.sum(true==predicted)/len(predicted)

In [120]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
df=pd.read_csv("/content/sports_survey_clean.csv")
columns=pd.read_excel("/content/sports_survey_clean_Column Labels.xlsx")

for i in range(95):
  df=df.rename(columns={df.columns[i]:columns.iloc[i,5]})

from sklearn.model_selection import train_test_split
fan_train, fan_test = train_test_split(df,
                                      shuffle = True,
                                      random_state = 555,
                                      test_size = .2)



In [121]:
list=['Went to a game','Watched a game at a sports bar','Placed a bet through a sportsbook/casino/bookie','Called into a sports radio show','Talked about games online','Listened to sports radio','Played in a fantasy sports league','Purchased a multi-game ticket package (e.g., season tickets, partial plan) for a professional sports team','Bet in a group pool, e.g., football squares, survivor leagues','Played daily fantasy']

In [122]:
correlation_matrix = fan_train.corr()

dfs = []

for i in range(10):
  target_correlations = correlation_matrix[list[i]]
  a=target_correlations.nlargest(10)
  index=a.index
  X_train=fan_train[[index[1],index[2],index[3],index[4],index[5],index[6],index[7],index[8]]]
  y_train=fan_train[[index[0]]]

  from sklearn.linear_model import LogisticRegression
  log_reg = LogisticRegression(penalty=None)
  #X_train=fan_train[[index[1],index[2],index[3],index[4],index[5],index[6],index[7],index[8]]]
  X_train=X_train.to_numpy()
  #y_train=fan_train['Went to a game']
  y_train=y_train.to_numpy()
  log_reg.fit(X_train,y_train)
  y_train_pred_log=log_reg.predict(X_train)
  y_train_pred_log=y_train_pred_log.reshape(8289,1)

  result = {"Model": "logistic", "column2": accuracy(y_train, y_train_pred_log),"y":list[i]}
  dfs.append(result)


  from sklearn.neighbors import KNeighborsClassifier
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import StandardScaler

  knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier(5))])

  knn_pipe.fit(X_train,
           y_train)

  y_train_pred_knn=knn_pipe.predict(X_train)
  y_train_pred_knn=y_train_pred_knn.reshape(8289,1)
  result = {"Model": "Knn", "column2": accuracy(y_train, y_train_pred_knn),"y":list[i]}
  dfs.append(result)

  from sklearn.tree import DecisionTreeClassifier
  from sklearn import tree

  tree_clf = DecisionTreeClassifier(max_depth=10,random_state= 216)

  ## fit the classifier
  tree_clf.fit(X_train, y_train)

  y_train_pred_dt=tree_clf.predict(X_train)
  y_train_pred_dt=y_train_pred_dt.reshape(8289,1)
  result = {"Model": "DT", "column2": accuracy(y_train, y_train_pred_dt),"y":list[i]}
  dfs.append(result)

  ## import model objects
  from sklearn.tree import DecisionTreeClassifier
  from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
  from sklearn.metrics import accuracy_score
  tree = DecisionTreeClassifier(max_depth = 10, random_state= 216)
  rf = RandomForestClassifier(n_estimators = 500,min_samples_leaf = 5,max_features = 2,bootstrap= True,max_samples = 500,random_state = 216)
  et = ExtraTreesClassifier(n_estimators = 500,min_samples_leaf = 5, max_features = 2, random_state = 216)

  tree.fit(X_train, y_train)

  ## rf and et may take slightly longer because we are fitting many trees
  rf.fit(X_train, y_train)
  et.fit(X_train, y_train)

  trees_pred=tree.predict(X_train)
  trees_pred=trees_pred.reshape(8289,1)
  rf_preds = rf.predict(X_train)
  rf_preds=rf_preds.reshape(8289,1)
  et_preds = et.predict(X_train)
  et_preds=et_preds.reshape(8289,1)

  result = {"Model": "tree", "column2": accuracy(y_train, trees_pred),"y":list[i]}
  dfs.append(result)

  result = {"Model": "rf", "column2": accuracy(y_train, rf_preds),"y":list[i]}
  dfs.append(result)

  result = {"Model": "et", "column2": accuracy(y_train, et_preds),"y":list[i]}
  dfs.append(result)

  from sklearn.ensemble import AdaBoostClassifier

  ## Import the base classifier
  from sklearn.tree import DecisionTreeClassifier


  ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=100,algorithm = 'SAMME',learning_rate = 0.5,random_state=123)
  ada_clf.fit(X_train, y_train)
  ada_pred=ada_clf.predict(X_train)
  ada_pred=ada_pred.reshape(8289,1)
  result = {"Model": "Ada", "column2": accuracy(y_train, ada_pred),"y":list[i]}
  dfs.append(result)

  from sklearn.ensemble import GradientBoostingRegressor

  n_trees = 200
  gb = GradientBoostingRegressor(max_depth=1,
                               n_estimators=n_trees)

  ## fit the booster
  gb.fit(X_train, y_train)

  gb_pred = gb.predict(X_train)
  gb_pred=np.where(gb_pred < 0.5, 0, 1)
  gb_pred=gb_pred.reshape(8289,1)
  result = {"Model": "GB", "column2": accuracy(y_train, gb_pred),"y":list[i]}
  dfs.append(result)


  # XGBoost
  import xgboost

  xgb_reg1 = xgboost.XGBRegressor(learning_rate=.5,max_depth=7,n_estimators=15)
  xgb_reg1.fit(X_train, y_train)

  xgb_pred = xgb_reg1.predict(X_train)
  xgb_pred=np.where(xgb_pred < 0.5, 0, 1)
  xgb_pred=xgb_pred.reshape(8289,1)

  result = {"Model": "GB", "column2": accuracy(y_train, xgb_pred),"y":list[i]}
  dfs.append(result)

















/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
<ipython-input-122-f9e026a22bdd>:64: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
<ipython-input-122-f9e026a22bdd>:65: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  et.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packag

In [123]:
dfs

[{'Model': 'logistic', 'column2': 0.7146821088189166, 'y': 'Went to a game'},
 {'Model': 'Knn', 'column2': 0.7716250452406804, 'y': 'Went to a game'},
 {'Model': 'DT', 'column2': 0.7616117746410906, 'y': 'Went to a game'},
 {'Model': 'tree', 'column2': 0.7616117746410906, 'y': 'Went to a game'},
 {'Model': 'rf', 'column2': 0.7205935577271082, 'y': 'Went to a game'},
 {'Model': 'et', 'column2': 0.7417058752563639, 'y': 'Went to a game'},
 {'Model': 'Ada', 'column2': 0.7169743032935215, 'y': 'Went to a game'},
 {'Model': 'GB', 'column2': 0.7158885269634455, 'y': 'Went to a game'},
 {'Model': 'GB', 'column2': 0.7785016286644951, 'y': 'Went to a game'},
 {'Model': 'logistic',
  'column2': 0.7133550488599348,
  'y': 'Watched a game at a sports bar'},
 {'Model': 'Knn',
  'column2': 0.7735553142719267,
  'y': 'Watched a game at a sports bar'},
 {'Model': 'DT',
  'column2': 0.7658342381469417,
  'y': 'Watched a game at a sports bar'},
 {'Model': 'tree',
  'column2': 0.7658342381469417,
  'y': 